In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm

Данные полученные в этом и следующем разделах были отброшены на ранних этапах работы

### Создание датасета из jsonов

In [2]:
def dataParse(dataframe, dump, channel):
    for message in dump:
        if 'reply_count' in message and 'subtype' not in message:
            dataframe.loc[len(df), ['channel', 'ThreadStarter', 'Question', 'Answers', 'reply_count', 'thread_ts']] = \
            [channel, message['user'], message['text'], '', message['reply_count'], message['thread_ts']]
        elif 'parent_user_id' in message:
            try:
                dataframe.loc[(dataframe['thread_ts'] == message['thread_ts']) & (dataframe['ThreadStarter'] == message['parent_user_id']), 'Answers']\
                += ("<BOA>" + message['text'] + "<EOA>")
            except KeyError:
                continue

In [3]:
df = pd.DataFrame(columns=['channel', 'ThreadStarter', 'Question', 'Answers', 'reply_count', 'thread_ts'])
for root, dirs, files in os.walk("opendatascience Slack export Mar 12 2015 - Jan 14 2020", topdown = False):
    for name in files:
        with open(os.path.join(root, name)) as json_data:
            dump = json.load(json_data)
            dataParse(df, dump, root.split('/')[-1])

df.head()

,channel,ThreadStarter,Question,Answers,reply_count,thread_ts
0,lang_go,U64GCA997,"Народ, кто знает, есть ли в GO библиотека для ...","<BOA>В python. Аналог pandas есть только в R, ...",5,1549648499.004600
1,lang_go,U0FEJNBGQ,"чо как, ml стало легче тут использовать?",<BOA>Деплоить тф модели относительно easy и по...,3,1574626852.009300
2,lang_go,U64GCA997,"Народ, какие у вам задавали вопросы на собесе ...",<BOA>Как насчет дженериков? :kekeke:<EOA><BOA>...,7,1575315642.018100
3,lang_go,U7KHK3TTP,"Блин, а это что за шляпа такая? `CGO_LDFLAGS_A...","<BOA>Стоило написать, сразу нашел\n<https://gi...",1,1534807275.000100
4,lang_go,U3B7T7V8W,Подскажите как вот эту штуку покрасивее сделат...,"<BOA>есть вариант, вызывать эти `go Action` в...",1,1538993224.000100


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83101 entries, 0 to 83100
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel        83101 non-null  object
 1   ThreadStarter  83101 non-null  object
 2   Question       83101 non-null  object
 3   Answers        83101 non-null  object
 4   reply_count    83101 non-null  object
 5   thread_ts      83101 non-null  object
dtypes: object(6)
memory usage: 6.9+ MB


In [26]:
df_t = df.copy()

In [27]:
df_t['reply_count'] = df_t['reply_count'].astype(int); df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83101 entries, 0 to 83100
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel        83101 non-null  object
 1   ThreadStarter  83101 non-null  object
 2   Question       83101 non-null  object
 3   Answers        83101 non-null  object
 4   reply_count    83101 non-null  int64 
 5   thread_ts      83101 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.4+ MB


In [32]:
df_t.loc[df_t['Answers'] == '', 'Answers'] = np.nan

In [42]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83101 entries, 0 to 83100
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel        83101 non-null  object
 1   ThreadStarter  83101 non-null  object
 2   Question       83101 non-null  object
 3   Answers        77474 non-null  object
 4   reply_count    83101 non-null  int64 
 5   thread_ts      83101 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.9+ MB


### Создание пар вопрос-ответ по всем данным

In [2]:
df = pd.read_parquet('data/QA_ODS_mod.parquet'); df.head()

,channel,ThreadStarter,Question,Answers,reply_count,thread_ts
0,lang_go,U64GCA997,"Народ, кто знает, есть ли в GO библиотека для ...","<BOA>В python. Аналог pandas есть только в R, ...",5,1549648499.004600
1,lang_go,U0FEJNBGQ,"чо как, ml стало легче тут использовать?",<BOA>Деплоить тф модели относительно easy и по...,3,1574626852.009300
2,lang_go,U64GCA997,"Народ, какие у вам задавали вопросы на собесе ...",<BOA>Как насчет дженериков? :kekeke:<EOA><BOA>...,7,1575315642.018100
3,lang_go,U7KHK3TTP,"Блин, а это что за шляпа такая? `CGO_LDFLAGS_A...","<BOA>Стоило написать, сразу нашел\n<https://gi...",1,1534807275.000100
4,lang_go,U3B7T7V8W,Подскажите как вот эту штуку покрасивее сделат...,"<BOA>есть вариант, вызывать эти `go Action` в...",1,1538993224.000100


In [3]:
clf_df = pd.DataFrame(columns=['Question', 'Answer', 'isAnswer'])

In [4]:
df = df.dropna(); df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77474 entries, 0 to 83099
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel        77474 non-null  object
 1   ThreadStarter  77474 non-null  object
 2   Question       77474 non-null  object
 3   Answers        77474 non-null  object
 4   reply_count    77474 non-null  int64 
 5   thread_ts      77474 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.1+ MB


In [5]:
for ind, row in tqdm(df.iterrows(), total=len(df)):
    #get real answers
    ans = row['Answers'].replace('<BOA>', '').split('<EOA>')
    #get rid of '' in answers
    if len(ans) > row['reply_count']:
        ans = ans[:-1]
    for a in ans:
        clf_df.loc[len(clf_df), ['Question', 'Answer', 'isAnswer']] = row['Question'], a, 1
    
    #get randow row from data
    r_row = df.sample()
    #get fake answers
    ans = r_row['Answers'].values[0].replace('<BOA>', '').split('<EOA>')
    #get rid of '' in answers
    if len(ans) > row['reply_count']:
        ans = ans[:-1]
    for a in ans:
        clf_df.loc[len(clf_df), ['Question', 'Answer', 'isAnswer']] = row['Question'], a, 0

100%|██████████| 77474/77474 [20:34:59<00:00,  1.05it/s]


In [6]:
clf_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320570 entries, 0 to 1320569
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Question  1320570 non-null  object
 1   Answer    1320570 non-null  object
 2   isAnswer  1320570 non-null  object
dtypes: object(3)
memory usage: 40.3+ MB


In [8]:
clf_df['isAnswer'].value_counts()

0    672587
1    647983
Name: isAnswer, dtype: int64

In [10]:
d = clf_df.sample(frac=1); d.head()

,Question,Answer,isAnswer
1055395,"Новые вакансии в <http://comma.ai|comma.ai>, в...",А обязательно приводить к одному размеру? Оно ...,0
392890,"Что думаете, будет шейкап?",<@U4B8MC0AD> Насколько я помню там предлагаютс...,0
1051930,Там главное всех unfollow нахер,Мало-мало человек фолловлю,1
892457,На recsys2017 кто-нибудь поехал?,и рандомщиков! =) ибо потом можно прорешать и ...,0
1179986,"Друг говорит, что на запрос инвайта нет ответа...",Еще набор утверждений:\n1) такой проблемы с по...,0


In [11]:
d.to_parquet('data/QA_bin.parquet', compression='brotli', index=False)

## Создание таблицы со всеми записями из jsonов
Эта таблица была взята за основу всей дальнешей работы, впоследствии все преобразования проводились на данных из неё

In [2]:
ofile_path = 'data/ods_slack_all.csv'

#### Таблица со всеми записями

In [3]:
import glob
idir = 'opendatascience Slack export Mar 12 2015 - Jan 14 2020'
lines = []
for channel_dir in glob.glob(f'{idir}/*'):
    channel_files = glob.glob(f'{channel_dir}/*.json')
    channel = os.path.basename(channel_dir)
    print(channel, len(channel_files))
    for ifile_path in channel_files:
        with open(ifile_path, encoding='utf-8') as ifile:
            data_json_list = json.load(ifile)
            data_json_new_list = []
            for x in data_json_list:
                x['channel'] = channel
                data_json_new_list.append(x)
            lines.extend(data_json_new_list)
df = pd.DataFrame(lines)
df.to_csv(ofile_path, encoding='utf-8')

lang_go 54
_meetings_dv 54
proj_kaggle_quest_challenge 20
_meetings_live 382
proj_kaggle_elo_merch 48
proj_yt8m 73
gis 390
proj_geohack 25
proj_kaggle_talkdata 63
tool_dvc 113
engineering-hiring 16
proj_retailhero 27
deep_learning 1605
ods_podcast 5
proj_nips_traffic 5
ods_sport 513
_meetings_norge 156
class_cs224n 1
_meetings_spb 1121
gan 8
chess 159
proj_kaggle_fish 212
proj_kaggle_airbus 78
sberbank_contest 398
proj_kaggle_bengali_classification 6
ods_policy_prior 17
_top_science 9
blackoxchallenge 4
edu_courses 1433
class_udacity_pytorch 103
_random_politics 416
edu_course_pgm 80
ods_site 108
proj_nips_autodl 4
blockchain 622
_meetings_minsk 499
_random_talks 1733
proj_kaggle_quickdraw 39
ucuracy_trainings 45
proj_kaggle_fashion 38
big_data 1201
meeting_nsfw_icml 27
ods_travel 93
proj_kaggle_petfinder 57
neurohack_heart 32
proj_comp_workshop 95
idrnd_challenge 44
ods_crowd 25
_meetings 1596
_meetings_tbilisi 18
sandbox 1
proj_kaggle_trackml 28
nlp 1438
proj_home_credit 71
neuroscie

In [3]:
df = pd.read_csv(ofile_path, dtype={'ts': str})
df['user_ts'] = df.apply(lambda x: f"{x['user']}_{x['ts']}", axis=1)
df.index = df['user_ts']
df_reply = df.query('reply_count > 0')
df_reply = df_reply[df_reply['subtype'].isna()]

In [4]:
%%time
def get_msg_ind(repl_msg_dict):
    return f"{repl_msg_dict['user']}_{repl_msg_dict['ts']}"

lines = []
#iterate through questions
for ind, msg in tqdm(df_reply.iterrows(), total=len(df_reply)):
    for repl_msg_dict in eval(msg['replies']):
        repl_msg_ind = get_msg_ind(repl_msg_dict)
        #search in full dataset
        #avoid bots' messages
        try:    
            repl_msg = df.loc[repl_msg_ind]
        except KeyError:
            print(repl_msg_dict)
            continue
        reacts = []
        #pass reactions to data if they exist
        try:
            for react in eval(repl_msg['reactions']):
                reacts.append({'name': react['name'], 'count': react['count']})

        except TypeError:
            reacts = repl_msg['reactions']
        lines.append({'channel': msg['channel'], 'question': msg['text'], 'answer': repl_msg['text'], 'reactions': reacts})

df_question_answer_text = pd.DataFrame(lines)

NE', 'ts': '1497801837.140618'}
{'user': 'B597E8DNE', 'ts': '1497801838.140674'}
{'user': 'B597E8DNE', 'ts': '1497801838.140700'}
  1%|          | 962/82727 [02:55<3:42:59,  6.11it/s]{'user': 'B597E8DNE', 'ts': '1500642047.088767'}
{'user': 'B597E8DNE', 'ts': '1500672257.500737'}
{'user': 'B597E8DNE', 'ts': '1500672257.500939'}
{'user': 'B597E8DNE', 'ts': '1500672257.501055'}
  1%|          | 1009/82727 [03:10<8:50:41,  2.57it/s]{'user': 'B597E8DNE', 'ts': '1499769598.662661'}
{'user': 'B597E8DNE', 'ts': '1499769599.662918'}
{'user': 'B597E8DNE', 'ts': '1499769680.688748'}
  1%|▏         | 1112/82727 [03:30<3:39:32,  6.20it/s]{'user': 'B597E8DNE', 'ts': '1503303452.000342'}
{'user': 'B597E8DNE', 'ts': '1503303453.000238'}
{'user': 'B597E8DNE', 'ts': '1503303454.000178'}
{'user': 'B597E8DNE', 'ts': '1503324283.000782'}
{'user': 'B597E8DNE', 'ts': '1503324285.000316'}
  2%|▏         | 1475/82727 [04:54<6:00:56,  3.75it/s]{'user': 'B597E8DNE', 'ts': '1498644382.815159'}
{'user': 'B597E8DN

In [5]:
df_question_answer_text.to_csv('data/qa_pairs.csv', encoding='utf-8')

In [11]:
df_question_answer_text.to_parquet('data/qa_pairs.parquet', compression='brotli', index=False)

### Парсим вопросы и ответы

In [2]:
df = pd.read_parquet('data/qa_pairs.parquet'); df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735931 entries, 0 to 735930
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   channel    735931 non-null  object
 1   question   731435 non-null  object
 2   answer     732830 non-null  object
 3   reactions  735931 non-null  object
dtypes: object(4)
memory usage: 22.5+ MB


In [3]:
df = df.dropna(); df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728406 entries, 0 to 735930
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   channel    728406 non-null  object
 1   question   728406 non-null  object
 2   answer     728406 non-null  object
 3   reactions  728406 non-null  object
dtypes: object(4)
memory usage: 27.8+ MB


Пробуем отделить знак вопроса от элемента ссылки

In [4]:
import re

LINK_PATTERN = re.compile('<https?://[^\"\s>]+>')

In [8]:
test_question = 'база действительно классная, но для ее анализа Юрий Евгеньевич применяет простейший PCA, потому что сам он человек не технический; я списался с ним, и он прислал мне эту базу, так что можно напустить на нее любые алгоритмы, начиная с кластеризации и заканчивая не знаю, чем; в общем, кто заинтересовался, пишите мне, устроим мозговой штурм; напоследок, свежее интервью с Юрием Евгеньевичем: <https://www.youtube.com/watch?v=2IXJBu_cAXU>'

text = re.sub(LINK_PATTERN, 'ссылка', test_question); text

'база действительно классная, но для ее анализа Юрий Евгеньевич применяет простейший PCA, потому что сам он человек не технический; я списался с ним, и он прислал мне эту базу, так что можно напустить на нее любые алгоритмы, начиная с кластеризации и заканчивая не знаю, чем; в общем, кто заинтересовался, пишите мне, устроим мозговой штурм; напоследок, свежее интервью с Юрием Евгеньевичем: ссылка'

In [13]:
df.loc[len(df)]

channel                                                  _jobs
question     Всем привет,\nОбъявление о наборе на PhD, подр...
answer       &gt; ML прикладная дисциплина, лучше интернату...
reactions    [{'name': 'kaggle', 'count': 8}, {'name': 'you...
Name: 728406, dtype: object

In [15]:
df_mod = pd.DataFrame(columns=df.columns)
for i, row in tqdm(df.iterrows(), total=len(df)):
    if '?' in re.sub(LINK_PATTERN, 'ссылка', row['question']):
        df_mod.loc[len(df_mod)] = row

print(f'Количество вопросов уменьшилось на {len(df)-len(df_mod)}')

100%|██████████| 728406/728406 [1:48:23<00:00, 112.00it/s]Количество вопросов уменьшилось на 323102



In [35]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405304 entries, 0 to 405303
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   channel    405304 non-null  object
 1   question   405304 non-null  object
 2   answer     405304 non-null  object
 3   reactions  405304 non-null  object
dtypes: object(4)
memory usage: 35.5+ MB


In [58]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_mod['channel'].value_counts())

career                                66754
_random_talks                         41052
deep_learning                         33058
theory_and_practice                   28396
lang_python                           21082
mlcourse_ai_news                      20872
nlp                                   17152
hardware                              14435
kaggle_crackers                        8280
edu_courses                            6665
ods_sport                              6441
mlcourse_ai                            5811
devops                                 5684
mltrainings_beginners                  5670
_meetings_sf                           5529
cv                                     5481
big_data                               5441
_random_b                              5311
datasets                               4217
blockchain                             3667
gnomiki                                3417
satellite_imaging                      3189
class_dl_cmu                    

Выбрасываем записи из каналов вакансий, потому что там нет вопросов

In [59]:
df_mod = df_mod[df_mod['channel'].str.find('jobs') == -1]

Смотрим, много ли останется ответов, если парсить по реакциям

In [60]:
df_mod = df_mod.reset_index()

In [61]:
indxs = []
r_cnt = 0
for i, row in tqdm(df_mod.iterrows(), total=len(df_mod)):
    if not 'nan' in row['reactions']:
        for reaction in eval(row['reactions']):
            r_cnt += 1
            name = reaction['name']
            if any([':tnx:' in name, ':heavy_plus_sign' in name, '+1' in name]):
                indxs.append(i)

print(f'\nОтветов с отмеченными реакциями всего {len(df_mod.loc[indxs])}, всего реакций {r_cnt}')

100%|██████████| 403332/403332 [00:31<00:00, 12894.01it/s]
Ответов с отмеченными реакциями всего 18363, всего реакций 123039



In [64]:
df_mod.drop('index', axis=1).to_parquet('data/qa_pairs_mod.parquet', compression='brotli', index=False)

### Размышления на счёт построения графа позитивных пар

In [2]:
df = pd.read_parquet('data/qa_pairs_mod.parquet'); df

,channel,question,answer,reactions
0,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","В python. Аналог pandas есть только в R, но та...",nan
1,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...","[{'name': 'heavy_plus_sign', 'count': 1}]"
2,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...",В Julia.,nan
3,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...",Вообще есть такая поделка - <https://github.co...,nan
4,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Ну и arrow есть, правда пока там мало что можн...",nan
...,...,...,...,...
403327,proj_kaggle_2sigma,"Ну что, как запасы попкорна на пол года? )",Вроде обещали обновлять результаты периодическ...,nan
403328,proj_kaggle_2sigma,"Ну что, как запасы попкорна на пол года? )","Интересно почистят ли лидерборд, както сомнива...",nan
403329,proj_kaggle_2sigma,"Ну что, как запасы попкорна на пол года? )","Вот не понятно, будут ли заморачиваться над пу...",nan
403330,proj_kaggle_2sigma,Нормализацию предиктов оставлять или ненадо? :...,Посмотри тут https: //www.kaggle.com/Questions...,nan


In [3]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['answer'].value_counts()[df['answer'].value_counts() > 2])

na.github.io/taiga_site/>                                                                                3
Все                                                                                                                            3
:wtf:                                                                                                                          3
Супер                                                                                                                          3
ну ладно                                                                                                                       3
вроде да                                                                                                                       3
Нет                                                                                                                            3
<@U0M39M6LS> ^^                                                                                                        

In [3]:
indxs = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    if any(['спасибо' in row['answer'].lower(), 'понятно' in row['answer'].lower(), 'отлично' in row['answer'].lower(), 'получилось' in row['answer'].lower(), 'работало' in row['answer'].lower()]):
        indxs.append(i)

print(f'\nОтветов выбросить {len(df.loc[indxs])}, количество записей станет {len(df)-len(df.loc[indxs])}')

100%|██████████| 403332/403332 [01:21<00:00, 4945.70it/s]

Ответов выбросить 21238, количество записей станет 382094


#### Выбираем самые чистые ответы с реакциями

In [3]:
indxs_clean = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    if not 'nan' in row['reactions']:
        for reaction in eval(row['reactions']):
            name = reaction['name']
            if all((any(['tnx' in name, 'heavy_plus_sign' in name, '+1' in name]), 
                    'спасибо' not in row['answer'].lower())):
                indxs_clean.append(i)

print(f'\nОтветов выбрaть {len(df.loc[indxs_clean])}, количество записей станет {len(df)-len(df.loc[indxs_clean])}')

100%|██████████| 403332/403332 [00:45<00:00, 8797.98it/s]

Ответов выбрaть 53462, количество записей станет 349870


Сохраняем, чтобы построить на них кластеры с помощью дефолтного USE

In [7]:
df.loc[indxs_clean].drop_duplicates('answer').to_parquet('data/qa_clean.parquet', compression='brotli', index=False)

In [4]:
df = df.loc[indxs_clean]
df = df.drop_duplicates('answer').reset_index().drop('index', axis=1); df

,channel,question,answer,reactions
0,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...","[{'name': 'heavy_plus_sign', 'count': 1}]"
1,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Зачем эта простыня текста. Ее традиционно никт...,"[{'name': 'tnx', 'count': 1}, {'name': 'heavy_..."
2,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",> в проект Data Science\nвыглядит как bullshit...,"[{'name': '+1', 'count': 1}]"
3,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",<@U64GCA997> сильно лучше :notbad:,"[{'name': 'tnx', 'count': 1}]"
4,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Что такое бустануть? Что за нужно уметь в них?...,"[{'name': 'heavy_plus_sign', 'count': 1}, {'na..."
...,...,...,...,...
49389,proj_kaggle_2sigma,"Как то можно писать в лог в процессе commit, ч...",можно `logging` юзать. тогда можно одно и тоже...,"[{'name': 'tnx', 'count': 1}]"
49390,proj_kaggle_2sigma,"Как то можно писать в лог в процессе commit, ч...",например так:\n```# Set up a logger to dump me...,"[{'name': '+1', 'count': 1}]"
49391,proj_kaggle_2sigma,"Ребят, а подскажите, я некоторое время назад с...","насколько я понимаю, после того как закончится...","[{'name': 'heavy_plus_sign', 'count': 2}]"
49392,proj_kaggle_2sigma,"Ребят, а подскажите, я некоторое время назад с...",1. Решения нет - это вообще бесспорно. 4 - наи...,"[{'name': '+1', 'count': 1}, {'name': 'heavy_p..."


In [5]:
df['question'].nunique()

21018

#### Отмечаем релевантные вопросу ответы

In [3]:
df = pd.read_parquet('data/not_load/qa_short.parquet')

In [4]:
graph = pd.DataFrame(columns=['question', 'answers', 'q_id', 'ans_ids'])
for i, row in tqdm(df.iterrows(), total=len(df)):
    if len(graph.loc[graph['question'] == row['question']]) == 0:
        graph.loc[len(graph), ['question', 'answers', 'q_id', 'ans_ids']] = row['question'], row['answer'], i, str(i)
    else:
        graph.loc[graph['question'] == row['question'], 'answers'] += '<EOA>'+row['answer']
        graph.loc[graph['question'] == row['question'], 'ans_ids'] += ','+str(i)

graph

100%|██████████| 49394/49394 [04:45<00:00, 173.10it/s]


,question,answers,q_id,ans_ids
0,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...",0,0
1,"Всем привет, нуждаюсь в помощи по составлению ...",Зачем эта простыня текста. Ее традиционно никт...,1,"1,2,3,4"
2,"и еще вопрос, нет ли веяний, что этот проект п...","кажется, что пока таких веяний нет, только рос...",5,"5,6"
3,Кто-нибудь здесь проходил\n<http://coursera.or...,"если говорить про идеоматичность, то я обычно ...",7,7
4,"Ahh folks, anyone is able to submit an lstm+en...",Fixed this!,8,8
...,...,...,...,...
21013,Привет!\nКогда-то давно написал маленькую библ...,интересный проект. но лучше перепишите на англ...,49385,49385
21014,<https://memepedia.ru/wp-content/uploads/2017/...,затвить что ли куда-нибудь - можно будет и там...,49386,"49386,49387"
21015,"Как то можно писать в лог в процессе commit, ч...","Можно писать отдельный файл в лог, который пот...",49388,"49388,49389,49390"
21016,"Ребят, а подскажите, я некоторое время назад с...","насколько я понимаю, после того как закончится...",49391,"49391,49392"


In [5]:
graph.to_parquet('data/not_load/qa_sim.parquet', compression='brotli', index=False)

### Выделяем короткие вопросы

In [2]:
df = pd.read_parquet('data/qa_clean.parquet')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49394 entries, 0 to 49393
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   channel    49394 non-null  object
 1   question   49394 non-null  object
 2   answer     49394 non-null  object
 3   reactions  49394 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [5]:
lens = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    lens.append(len(row['question'].split(' ')))
max(lens)

49394it [00:06, 7455.43it/s]


1800

In [6]:
from collections import Counter

c = Counter(lens)

In [23]:
c.most_common

387

In [22]:
c.most_common(100)

[(15, 1149),
 (16, 1023),
 (21, 1009),
 (19, 992),
 (18, 988),
 (20, 987),
 (11, 954),
 (12, 944),
 (22, 923),
 (13, 904),
 (14, 895),
 (17, 881),
 (23, 823),
 (24, 816),
 (10, 804),
 (29, 781),
 (8, 776),
 (32, 775),
 (33, 772),
 (26, 767),
 (9, 761),
 (27, 758),
 (34, 700),
 (25, 692),
 (28, 680),
 (31, 674),
 (7, 661),
 (30, 661),
 (36, 625),
 (48, 582),
 (35, 581),
 (38, 569),
 (41, 564),
 (37, 563),
 (39, 562),
 (6, 510),
 (42, 502),
 (40, 498),
 (45, 471),
 (47, 453),
 (43, 445),
 (5, 439),
 (44, 434),
 (46, 398),
 (52, 393),
 (51, 386),
 (58, 371),
 (53, 368),
 (49, 367),
 (54, 366),
 (57, 345),
 (50, 327),
 (61, 326),
 (59, 320),
 (55, 311),
 (4, 308),
 (64, 301),
 (70, 290),
 (62, 289),
 (68, 281),
 (56, 265),
 (71, 257),
 (66, 254),
 (60, 251),
 (63, 249),
 (67, 234),
 (74, 215),
 (75, 210),
 (85, 197),
 (78, 196),
 (88, 192),
 (69, 188),
 (65, 182),
 (73, 180),
 (72, 175),
 (89, 169),
 (90, 164),
 (3, 162),
 (79, 162),
 (98, 161),
 (76, 155),
 (81, 153),
 (83, 150),
 (87, 14

Выбираем максимальное число слов = 25

In [20]:
ids = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    if len(row['question'].split(' ')) <= 25:
        ids.append(i)

df.loc[ids].info()

100%|██████████| 49394/49394 [00:06<00:00, 7749.27it/s]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18487 entries, 5 to 49393
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   channel    18487 non-null  object
 1   question   18487 non-null  object
 2   answer     18487 non-null  object
 3   reactions  18487 non-null  object
dtypes: object(4)
memory usage: 722.1+ KB


In [21]:
df.to_parquet('data/qa_short.parquet', compression='brotli', index=False)

### Создаём таблицу для подбора по ней пар похожих вопросов

In [2]:
df = pd.read_parquet('data/qa_clean.parquet'); df.head()

,channel,question,answer,reactions
0,lang_go,"Народ, кто знает, есть ли в GO библиотека для ...","Шатать таблички в го это, как летать на дачу н...","[{'name': 'heavy_plus_sign', 'count': 1}]"
1,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Зачем эта простыня текста. Ее традиционно никт...,"[{'name': 'tnx', 'count': 1}, {'name': 'heavy_..."
2,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",> в проект Data Science\nвыглядит как bullshit...,"[{'name': '+1', 'count': 1}]"
3,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",<@U64GCA997> сильно лучше :notbad:,"[{'name': 'tnx', 'count': 1}]"
4,lang_go,"Всем привет, нуждаюсь в помощи по составлению ...",Что такое бустануть? Что за нужно уметь в них?...,"[{'name': 'heavy_plus_sign', 'count': 1}, {'na..."


#### Создаем новую таблицу

In [3]:
channels = ['analytiсs', 'audio_and_speech', 'bayesian', 'big_data', 'bioinformatics', 'blockchain', 'cloud', 'datasets',
            'deep_learning', 'devops', 'gan', 'gis', 'lang_cpp', 'lang_go', 'lang_javascript', 'lang_julia', 'lang_python'
            'lang_r', 'lang_scala', 'math', 'nlp', 'recommender_systems', 'reinforcement_learning', 'risk_modelling', 
            'satellite_imaging', 'trading', 'visualization']

In [4]:
df_new = pd.DataFrame(columns=['question', 'similiar_questions_ids_in_clean_df', 'question_ids_in_clean_df', 'cheched'])

In [5]:
df = df[df['channel'].isin(channels)]

In [6]:
df_new['question'] = df['question'].unique()
df_new['checked'] = False

In [7]:
for i in tqdm(range(len(df_new)), total=len(df_new), desc="Сопоставляем индексы новой таблицы с индексами из qa_clean"):
    df_new.loc[i, 'question_ids_in_clean_df'] = str(df.index[df['question'] == df_new.loc[i, 'question']].tolist()).strip('[]')

Сопоставляем индексы новой таблицы с индексами из qa_clean: 100%|██████████| 5161/5161 [00:09<00:00, 536.60it/s]


In [8]:
df_new.head()

,question,similiar_questions_ids_in_clean_df,question_ids_in_clean_df,cheched,checked
0,"Народ, кто знает, есть ли в GO библиотека для ...",NaN,0,NaN,False
1,"Всем привет, нуждаюсь в помощи по составлению ...",NaN,"1, 2, 3, 4",NaN,False
2,"и еще вопрос, нет ли веяний, что этот проект п...",NaN,"5, 6",NaN,False
3,Кто-нибудь здесь проходил\n<http://coursera.or...,NaN,7,NaN,False
4,Кто-нибудь знает готовые реализации для обнару...,NaN,66,NaN,False


In [9]:
df_new.to_parquet('data/qq_sim.parquet', compression='brotli', index=False)

## Делаем датасет для классификации тем

In [2]:
df = pd.read_parquet('data/qa_pairs.parquet').dropna(); df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728406 entries, 0 to 735930
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   channel    728406 non-null  object
 1   question   728406 non-null  object
 2   answer     728406 non-null  object
 3   reactions  728406 non-null  object
dtypes: object(4)
memory usage: 27.8+ MB


In [3]:
import re

CHANNEL_TAG = re.compile('#\w{2,}')
LINK_PATTERN = re.compile('<https?://[^\"\s>]+>')
test_question = '#_nlp kadf'

text = re.findall(CHANNEL_TAG, test_question); len(text)

1

Убираем вопросы с перенаправлением в треде и оставляем только со знаками вопроса

In [7]:
df_mod = pd.DataFrame(columns=df.columns)
for question in tqdm(df['question'].unique(), total=len(df['question'].unique())):
    answers = df[df['question'] == question]['answer'].to_numpy()
    channel = df[df['question'] == question]['channel'].to_numpy()[0]
    reactions = df[df['question'] == question]['reactions'].to_numpy()
    for i, answer in enumerate(answers):
        if '?' in re.sub(LINK_PATTERN, 'ссылка', question) and len(re.findall(CHANNEL_TAG, answer)) == 0 and len(question.split(' '))<=512:
            df_mod.loc[len(df_mod)] = channel, question, answer, reactions[i]

print(f'\nКоличество вопросов уменьшилось на {len(df)-len(df_mod)}')

100%|██████████| 81238/81238 [4:03:19<00:00,  5.56it/s]
Количество вопросов уменьшилось на 330114



In [8]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398292 entries, 0 to 398291
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   channel    398292 non-null  object
 1   question   398292 non-null  object
 2   answer     398292 non-null  object
 3   reactions  398292 non-null  object
dtypes: object(4)
memory usage: 15.2+ MB


In [10]:
df_mod.to_parquet('data/qa_classif.parquet', compression='brotli', index=False)

Оставляем короткие вопросы

In [10]:
df = pd.read_parquet('data/train.parquet')

In [11]:
df.head()

,question,_call_4_collaboration,_jobs,_random_b,_random_politics,_random_talks,article_essence,audio_and_speech,bayesian,big_data,...,sequences_series,theory_and_practice,tool_albumentations,tool_catalyst,tool_catboost,tool_dvc,trading,visualization,welcome,writing_n_presenting
0,за hw9/10 баллы в рейтинг будут?,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,По поводу кассандры – а как ее разворачивать в...,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,При покупке на localbitcoin есть ли какие-то г...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Что посоветуете брать, поддержаную gtx 1080 ti...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Всем привет. Может кто-нибудь встречал описани...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    if len(row['question']) > 1024:
        df.loc[i, 'question'] = row['question'][:1024]

longer = 0
for i, row in tqdm(df.iterrows(), total=len(df)):
    if len(row['question']) > 1024:
        longer += 1

print('')
print(f'Questions longer then 1024 characters are {longer}')

100%|██████████| 215054/215054 [00:15<00:00, 14230.94it/s]
Questions longer then 1024 characters are 0



In [6]:
(len(df)-shorter)/len(df) * 100

5.260539213406865

In [14]:
df.to_parquet('data/tr_train.parquet', compression='brotli', index=False)

In [8]:
max_len = 0
max2_len = 0
max3_len = 0
n_max, n_max2, n_max3 = 0, 0, 0
for i, row in tqdm(df.iterrows(), total=len(df)):
    flag = False
    l = len(row['question'])
    if l > max_len:
        flag = True
    elif l == max_len:
        n_max += 1
    elif l == max2_len:
        n_max2 += 1
    elif l == max3_len:
        n_max3 += 1

    if flag:
        max3_len = max2_len
        max2_len = max_len
        max_len = l
        n_max3 = n_max2
        n_max2 = n_max
        n_max = 0

print('')
print(f'There are {n_max} questions of len = {max_len}\nThere are {n_max2} questions of len = {max2_len}\nThere are {n_max3} questions of len = {max3_len}')

100%|██████████| 215054/215054 [00:17<00:00, 12538.81it/s]
There are 9 questions of len = 3962
There are 62 questions of len = 3878
There are 44 questions of len = 3099



### Смотрим call_for_collaboration

In [5]:
df = pd.read_parquet('data/test.parquet')
c4c = df[df['_call_4_collaboration'] == 1].reset_index()

In [12]:
c4c.loc[6, 'question']

'Привет! А кто бы согласился часов на 40 учебный курс по машинному обучению для аудиторов провести? Сложность основная в том, что контингент довольно разнородный, около 50 человек, от вчерашних студентов до бабулек. Поэтому желателен бОльший уклон в прикладную часть, чем в теорию. Территориально - Москва. Какая именно компания - в личном общении.\nОб оплате готов в достаточно широких пределах обсуждать (от 10К/день, верхней границы нет).'

In [13]:
df = pd.read_parquet('data/train.parquet')
c4c_t = df[df['_call_4_collaboration'] == 1].reset_index()

In [15]:
c4c_t.loc[6, 'question']

'Всем привет.\nесть тема, связанная с определение мест в фильмах/передачах, в которые удобнее всего вставлять рекламные блоки (можно вставлять в сцену где нет диалога, нет экшна, в идеале там где картинка движется монотонно или экран темнеет).\nЕсть ли специалисты, которые умеют решать подобные задачи?'